# The Power of Incremental Improvement — Try it in PyTorch

This is an **optional** hands-on companion to [Chapter 2](https://robennals.github.io/ai-explained/02-optimization). You'll implement random search, compute gradients with PyTorch's autograd, and watch gradient descent converge.

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

## Random Search

Our goal is to find the input value that makes a function output the smallest possible number. We call that output the **loss** — it measures "how wrong" we are. A loss of 0 means perfect; bigger means worse. The process of finding the input that minimizes the loss is called **optimization**.

In [ ]:
# Goal: find the x that minimizes f(x) = (x - 3)² + 1
# Strategy: start somewhere, try random small changes, keep improvements

def f(x):
    return (x - 3)**2 + 1

x = torch.tensor(0.0)
best_loss = f(x)
history = [x.item()]

for step in range(200):
    delta = torch.randn(1).item() * 0.5
    x_new = x + delta
    new_loss = f(x_new)
    
    if new_loss < best_loss:
        x = x_new
        best_loss = new_loss
    history.append(x.item())

plt.figure(figsize=(8, 3))
plt.plot(history)
plt.axhline(y=3.0, color='r', linestyle='--', alpha=0.5, label='True minimum (x=3)')
plt.xlabel("Step")
plt.ylabel("x value")
plt.title("Random search: stumbling toward the answer")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print(f"Found x = {x.item():.4f}, f(x) = {f(x).item():.4f}  (true minimum: x=3, f(x)=1)")

## Smoothness and Gradients

A **gradient** is just the slope of a function — it tells you which direction is "downhill." If you're standing on a hillside, the gradient points toward the steepest uphill direction. To minimize the loss, you walk the opposite way.

`requires_grad=True` tells PyTorch: "keep track of every math operation on this number, so you can compute the slope later." Calling `.backward()` then calculates the gradient automatically. How this works internally (backpropagation) is covered in Chapter 3 — for now, just think of it as PyTorch doing the calculus for you.

In [ ]:
# PyTorch can automatically compute gradients (slopes)!
# This is called "autograd" — automatic differentiation

x = torch.tensor(0.0, requires_grad=True)

# Compute the function
loss = (x - 3)**2 + 1

# Compute the gradient (how does loss change when x changes?)
loss.backward()

print(f"x = {x.item():.1f}")
print(f"f(x) = {loss.item():.1f}")
print(f"gradient df/dx = {x.grad.item():.1f}")
print(f"\nThe gradient is {x.grad.item():.1f}, meaning:")
print(f"  - The function is {'decreasing' if x.grad.item() < 0 else 'increasing'} at x={x.item()}")
print(f"  - To decrease f(x), move x in the {'positive' if x.grad.item() < 0 else 'negative'} direction")

# Note: autograd uses backpropagation internally — we'll cover that in Chapter 3

## Gradient Descent

**Gradient descent** repeats a simple loop: compute the gradient, take a small step downhill, repeat. The **learning rate** controls how big each step is — too big and you overshoot, too small and it takes forever.

In [ ]:
# Gradient descent: follow the slope downhill, step by step

learning_rate = 0.1
x = torch.tensor(0.0, requires_grad=True)
history_x = [x.item()]
history_loss = []

for step in range(50):
    loss = (x - 3)**2 + 1
    history_loss.append(loss.item())
    
    loss.backward()
    
    with torch.no_grad():
        x -= learning_rate * x.grad
    
    x.grad.zero_()
    history_x.append(x.item())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

ax1.plot(history_x)
ax1.axhline(y=3.0, color='r', linestyle='--', alpha=0.5, label='Target (x=3)')
ax1.set_xlabel("Step")
ax1.set_ylabel("x value")
ax1.set_title("x converges to the minimum")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history_loss)
ax2.set_xlabel("Step")
ax2.set_ylabel("Loss")
ax2.set_title("Loss decreases smoothly")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print(f"Final: x = {x.item():.6f}, loss = {((x.item()-3)**2+1):.6f}")

## Random Search vs Gradient Descent

In [ ]:
# Compare random search vs gradient descent on a 2D problem
# f(x, y) = (x - 2)² + (y + 1)² + 3

def f_2d(x, y):
    return (x - 2)**2 + (y + 1)**2 + 3

# --- Random search ---
torch.manual_seed(42)
rx, ry = torch.tensor(0.0), torch.tensor(0.0)
best_loss = f_2d(rx, ry)
random_path = [(rx.item(), ry.item())]

for _ in range(200):
    dx, dy = torch.randn(2) * 0.3
    nx, ny = rx + dx, ry + dy
    nl = f_2d(nx, ny)
    if nl < best_loss:
        rx, ry, best_loss = nx, ny, nl
    random_path.append((rx.item(), ry.item()))

# --- Gradient descent ---
gx = torch.tensor(0.0, requires_grad=True)
gy = torch.tensor(0.0, requires_grad=True)
grad_path = [(gx.item(), gy.item())]
lr = 0.1

for _ in range(50):
    loss = f_2d(gx, gy)
    loss.backward()
    with torch.no_grad():
        gx -= lr * gx.grad
        gy -= lr * gy.grad
    gx.grad.zero_()
    gy.grad.zero_()
    grad_path.append((gx.item(), gy.item()))

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

for ax, path, title, color in [
    (ax1, random_path, "Random search (200 steps)", "blue"),
    (ax2, grad_path, "Gradient descent (50 steps)", "red"),
]:
    xs, ys = zip(*path)
    ax.plot(xs, ys, '-o', markersize=2, color=color, alpha=0.6)
    ax.plot(xs[0], ys[0], 'ko', markersize=8, label='Start')
    ax.plot(2, -1, 'r*', markersize=15, label='True minimum')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-3, 2)
    ax.set_title(title)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

rp = random_path[-1]
print(f"Random search: ({rp[0]:.3f}, {rp[1]:.3f}), loss = {f_2d(torch.tensor(rp[0]), torch.tensor(rp[1])):.4f}")
print(f"Gradient descent: ({gx.item():.3f}, {gy.item():.3f}), loss = {f_2d(gx, gy).item():.4f}")
print(f"\nGradient descent found the minimum in 50 steps. Random search used 200 and was less precise.")

---

*This notebook accompanies [Chapter 2: The Power of Incremental Improvement](https://robennals.github.io/ai-explained/02-optimization). The interactive widgets in the web version let you explore these concepts visually.*